In [3]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import codecs
import nltk
import re

from langchain.text_splitter import RecursiveCharacterTextSplitter
nltk.download('punkt')
# Chat specific components
from langchain.memory import ConversationBufferMemory

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [4]:
#API key congfiguration
# create .env file and give api key: OPENAI_API_KEY="sk-"
from decouple import config
import os 
%load_ext dotenv
%dotenv
openai_api_key=os.getenv("OPENAI_API_KEY")


In [6]:
template = """
You are a chatbot.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
llm_chain = LLMChain(
    llm=OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY")), 
    prompt=prompt, 
    verbose=True, 
    memory=memory
    
)

In [9]:

with codecs.open('data/jobs_data.csv', 'r',encoding="utf8") as f:
    for line in f:
        tweet = f.readlines()
        tokenized_sents = [nltk.word_tokenize(i) for i in tweet]
        
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=2,
    length_function = len,
    separators=['\n\n','\n',' ','']
)

i=re.sub("[0-9]+", "X", str(tokenized_sents))
texts = text_splitter.create_documents(i)
# print(texts[0])
# print(texts[1])
# print(text_splitter.split_text(i))
# print("________________________________________________________")

In [11]:
#convert dat ato vector and use pinecone.
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-l6-v2")

In [12]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(text_splitter.split_text(i),embeddings)

AttributeError: 'str' object has no attribute 'page_content'

In [10]:
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([text_splitter.split_text(i)])
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

AttributeError: 'str' object has no attribute 'load'